<a href="https://colab.research.google.com/github/shubhangiisankar/AICTE-Internships-2025-June_2025-Water-Quality-Prediction/blob/main/DrugDrugInteraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import torch
print("GPU available:", torch.cuda.is_available())


GPU available: True


In [6]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
!ls /content/drive/MyDrive


'ATS_Shubhangii_Resumee (2).pdf'      'Colab Notebooks'
'ATS_Shubhangii_Resumee (5)_wff.pdf'  'drug_with_unique_side_effects (1).xlsx'
 ATS_Shubhangii_Resumee.pdf	       resume_cjk.pdf
 ATS_shubhangi_Resume.pdf	      'Shubhangi_Isankar_new .pdf'


In [8]:
!pip install -q transformers datasets accelerate scikit-learn


In [9]:
import pandas as pd
import torch
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer
)
from sklearn.model_selection import train_test_split


In [59]:
data_path = "/content/drive/MyDrive/drug_with_unique_side_effects (1).xlsx"
df = pd.read_excel(data_path)

df.head()

,drugbank_id,name,description,type,side_effects
0,DB00001,Lepirudin,Lepirudin is identical to natural hirudin exce...,biotech,"Back pain, Sore throat, Cough, Tingling, Depre..."
1,DB00002,Cetuximab,Cetuximab is an epidermal growth factor recept...,biotech,"Muscle pain, Chills, Nausea, Low blood pressur..."
2,DB00003,Dornase alfa,Dornase alfa is a biosynthetic form of human d...,biotech,"Abdominal pain, Numbness, Photosensitivity, Fe..."
3,DB00004,Denileukin diftitox,A recombinant DNA-derived cytotoxic protein co...,biotech,"Fatigue, Weakness, Kidney dysfunction, Diarrhe..."
4,DB00005,Etanercept,Dimeric fusion protein consisting of the extra...,biotech,"Headache, Hives, Weakness, Weight gain, Shortn..."


In [63]:
df.columns


Index(['drugbank_id', 'name', 'description', 'type', 'side_effects'], dtype='object')

In [56]:
import numpy as np

labels = df['label']   # or your actual label column
unique, counts = np.unique(labels, return_counts=True)
print(dict(zip(unique, counts)))


{np.float64(0.0): np.int64(10562)}


In [11]:
print(df.columns)
print("Total rows:", len(df))


Index(['drugbank_id', 'name', 'description', 'type', 'side_effects'], dtype='object')
Total rows: 10562


In [12]:
df["text"] = (
    "Drug name: " + df["name"].astype(str) +
    ". Known side effects include: " + df["side_effects"].astype(str) + "."
)

df = df[["text"]]
df.head()


,text
0,Drug name: Lepirudin. Known side effects inclu...
1,Drug name: Cetuximab. Known side effects inclu...
2,Drug name: Dornase alfa. Known side effects in...
3,Drug name: Denileukin diftitox. Known side eff...
4,Drug name: Etanercept. Known side effects incl...


In [25]:
df["label"] = 0.0   # ✅ FLOAT label



In [26]:
train_df, val_df = train_test_split(
    df,
    test_size=0.2,
    random_state=42
)

train_dataset = Dataset.from_pandas(train_df)
val_dataset   = Dataset.from_pandas(val_df)


In [27]:
model_name = "microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract"

tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=1
)


Loading weights:   0%|          | 0/199 [00:00<?, ?it/s]

BertForSequenceClassification LOAD REPORT from: microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract
Key                                        | Status     | 
-------------------------------------------+------------+-
cls.predictions.transform.dense.weight     | UNEXPECTED | 
cls.predictions.transform.LayerNorm.bias   | UNEXPECTED | 
cls.predictions.decoder.weight             | UNEXPECTED | 
cls.seq_relationship.weight                | UNEXPECTED | 
cls.predictions.transform.dense.bias       | UNEXPECTED | 
cls.predictions.bias                       | UNEXPECTED | 
cls.predictions.transform.LayerNorm.weight | UNEXPECTED | 
cls.predictions.decoder.bias               | UNEXPECTED | 
cls.seq_relationship.bias                  | UNEXPECTED | 
classifier.weight                          | MISSING    | 
classifier.bias                            | MISSING    | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
- MISSING	:

In [35]:
train_dataset = Dataset.from_pandas(train_df)
val_dataset   = Dataset.from_pandas(val_df)


In [36]:
def tokenize(batch):
    return tokenizer(
        batch["text"],
        padding="max_length",
        truncation=True,
        max_length=128
    )

train_dataset = train_dataset.map(
    tokenize,
    batched=True,
    remove_columns=["text"]  # ✅ remove ONLY text
)

val_dataset = val_dataset.map(
    tokenize,
    batched=True,
    remove_columns=["text"]  # ✅ remove ONLY text
)


Map:   0%|          | 0/8449 [00:00<?, ? examples/s]

Map:   0%|          | 0/2113 [00:00<?, ? examples/s]

In [37]:
print(train_dataset.column_names)


['label', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask']


In [38]:
train_dataset = train_dataset.remove_columns(["__index_level_0__"])
val_dataset   = val_dataset.remove_columns(["__index_level_0__"])


In [39]:
print(train_dataset.column_names)


['label', 'input_ids', 'token_type_ids', 'attention_mask']


In [40]:
train_dataset.set_format(
    type="torch",
    columns=["input_ids", "attention_mask", "label"]
)

val_dataset.set_format(
    type="torch",
    columns=["input_ids", "attention_mask", "label"]
)


In [17]:
!pip install -U transformers


In [18]:
import transformers
print(transformers.__version__)


5.1.0


In [28]:
training_args = TrainingArguments(
    output_dir=output_dir,

    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=1,

    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,

    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    learning_rate=2e-5,
    weight_decay=0.01,

    logging_dir=f"{output_dir}/logs",
    report_to="none"
)


`logging_dir` is deprecated and will be removed in v5.2. Please set `TENSORBOARD_LOGGING_DIR` instead.


In [42]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)


In [43]:
trainer.train()


Epoch,Training Loss,Validation Loss
1,0.002285,0.000056
2,0.000773,0.000045
3,0.000450,0.000011


Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

There were missing keys in the checkpoint model loaded: ['bert.embeddings.LayerNorm.weight', 'bert.embeddings.LayerNorm.bias', 'bert.encoder.layer.0.attention.output.LayerNorm.weight', 'bert.encoder.layer.0.attention.output.LayerNorm.bias', 'bert.encoder.layer.0.output.LayerNorm.weight', 'bert.encoder.layer.0.output.LayerNorm.bias', 'bert.encoder.layer.1.attention.output.LayerNorm.weight', 'bert.encoder.layer.1.attention.output.LayerNorm.bias', 'bert.encoder.layer.1.output.LayerNorm.weight', 'bert.encoder.layer.1.output.LayerNorm.bias', 'bert.encoder.layer.2.attention.output.LayerNorm.weight', 'bert.encoder.layer.2.attention.output.LayerNorm.bias', 'bert.encoder.layer.2.output.LayerNorm.weight', 'bert.encoder.layer.2.output.LayerNorm.bias', 'bert.encoder.layer.3.attention.output.LayerNorm.weight', 'bert.encoder.layer.3.attention.output.LayerNorm.bias', 'bert.encoder.layer.3.output.LayerNorm.weight', 'bert.encoder.layer.3.output.LayerNorm.bias', 'bert.encoder.layer.4.attention.output.La

TrainOutput(global_step=3171, training_loss=0.0018843861454783536, metrics={'train_runtime': 640.9, 'train_samples_per_second': 39.549, 'train_steps_per_second': 4.948, 'total_flos': 1667254010317056.0, 'train_loss': 0.0018843861454783536, 'epoch': 3.0})

In [44]:
trainer.save_model(output_dir)
tokenizer.save_pretrained(output_dir)


Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

('/content/drive/MyDrive/pubmedbert_single_drug/tokenizer_config.json',
 '/content/drive/MyDrive/pubmedbert_single_drug/tokenizer.json')

In [45]:
eval_results = trainer.evaluate()

print("Evaluation Results:")
for key, value in eval_results.items():
    print(f"{key}: {value}")


Evaluation Results:
eval_loss: 1.1145241842314135e-05
eval_runtime: 13.7839
eval_samples_per_second: 153.295
eval_steps_per_second: 19.225
epoch: 3.0


In [46]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix


In [48]:
predictions = trainer.predict(val_dataset)

logits = predictions.predictions
labels = predictions.label_ids

y_pred = np.argmax(logits, axis=1)
y_true = labels


In [49]:
accuracy = accuracy_score(y_true, y_pred)

precision, recall, f1, _ = precision_recall_fscore_support(
    y_true, y_pred, average='weighted'
)

print("Evaluation Metrics:")
print(f"Accuracy : {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall   : {recall:.4f}")
print(f"F1 Score : {f1:.4f}")


Evaluation Metrics:
Accuracy : 1.0000
Precision: 1.0000
Recall   : 1.0000
F1 Score : 1.0000


In [50]:
cm = confusion_matrix(y_true, y_pred)
print("Confusion Matrix:")
print(cm)


Confusion Matrix:
[[2113]]


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:407: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(


In [51]:
from sklearn.metrics import classification_report

print(classification_report(y_true, y_pred))


              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      2113

    accuracy                           1.00      2113
   macro avg       1.00      1.00      1.00      2113
weighted avg       1.00      1.00      1.00      2113



In [65]:
from transformers import AutoTokenizer, AutoModel
import torch

model_path = "/content/drive/MyDrive/pubmedbert_single_drug"

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModel.from_pretrained(model_path)

model.eval()  # VERY important


Loading weights:   0%|          | 0/199 [00:00<?, ?it/s]

BertModel LOAD REPORT from: /content/drive/MyDrive/pubmedbert_single_drug
Key               | Status     |  | 
------------------+------------+--+-
classifier.weight | UNEXPECTED |  | 
classifier.bias   | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
  

In [66]:
def get_drug_embedding(text):
    inputs = tokenizer(
        text,
        return_tensors="pt",
        truncation=True,
        padding=True,
        max_length=128
    )

    with torch.no_grad():
        outputs = model(**inputs)

    # CLS token embedding
    cls_embedding = outputs.last_hidden_state[:, 0, :]

    return cls_embedding.squeeze().numpy()


In [67]:
def build_drug_text(row):
    return (
        f"Drug name: {row['name']}. "
        f"Description: {row['description']}. "
        f"Known side effects include: {row['side_effects']}."
    )


In [ ]:
import numpy as np

drug_embeddings = {}
for _, row in df.iterrows():
    drug_text = build_drug_text(row)
    embedding = get_drug_embedding(drug_text)
    drug_embeddings[row["name"]] = embedding


In [ ]:
import pickle

with open("/content/drive/MyDrive/drug_embeddings.pkl", "wb") as f:
    pickle.dump(drug_embeddings, f)
